In [1]:
from glob import glob
from tqdm import tqdm
from pathlib import Path
import zipfile
import sys
import dotenv
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
tf.compat.v1.enable_eager_execution()

dotenv.load_dotenv(override=True)
WORK = os.getenv('WORK')
sys.path.insert(0, f'{WORK}/ADA_Project')
sys.path.insert(0, f'{WORK}/ADA_Project/StyleGAN2-ADA')

import DeepFAMS

WORK, PROJ_DIR = DeepFAMS.utils.set_env()

Tensorflow version: 1.14.0


In [2]:
RAW_IMGS_DIR, RESIZED_IMGS_DIR, DATA_CUSTOM_DIR, TRAIN_RUNS_DIR = DeepFAMS.utils.return_dirs(
    PROJ_DIR, 'FFHQ')

In [ ]:
# ! $WORK/.conda/envs/ada-env/bin/kaggle datasets download -d \
#     arnaud58/flickrfaceshq-dataset-ffhq -p $WORK/ADA_Project/datasets

In [12]:
# downloaded_file = f'{Path(RAW_IMGS_DIR).parent}/flickrfaceshq-dataset-ffhq.zip'

# with zipfile.ZipFile(downloaded_file) as zf:    
#     for member in tqdm(zf.infolist(), desc='Extracting'):
#         try:
#             zf.extract(member, RAW_IMGS_DIR)
#         except zipfile.error as e:
#             pass

Extracting: 100%|██████████| 52001/52001 [09:29<00:00, 91.36it/s] 


In [14]:
raw_imgs = glob(f'{RAW_IMGS_DIR}/*')
print(len(raw_imgs))

52001


In [15]:
# for x in tqdm(raw_imgs):
#     DeepFAMS.preprocessing.resize_imgs(x, (256, 256), RESIZED_IMGS_DIR)

100%|██████████| 52001/52001 [30:21<00:00, 28.55it/s] 


In [16]:
print(f'Raw: {len(raw_imgs)}, Resized: {len(glob(f"{RESIZED_IMGS_DIR}/*"))}')

Raw: 52001, Resized: 52001


In [17]:
# DeepFAMS.preprocessing.tf_record_exporter(
#     tfrecord_dir=DATA_CUSTOM_DIR, image_dir=RESIZED_IMGS_DIR, shuffle=1)

Loading images from "/work/chaselab/malyetama/ADA_Project/datasets/FFHQ_resized"
Creating dataset "/work/chaselab/malyetama/ADA_Project/datasets/FFHQ_custom"
Added 52001 images.                     


In [ ]:
DeepFAMS.utils.executePopen(f'''#!/bin/bash
module load anaconda
module load compiler/gcc/4.7
module load cuda
$WORK/.conda/envs/ada-env/bin/python $WORK/ADA_Project/StyleGAN2-ada/train.py \
--outdir={TRAIN_RUNS_DIR} \
--gpus=8 \
--data={DATA_CUSTOM_DIR} \
--snap=1 \
--kimg=1''', PROJ_DIR)

In [ ]:
for num in range(-1, -10, -1):
    files = DeepFAMS.utils.last_snap(num, TRAIN_RUNS_DIR)
    if files != []:
        break

latest_snap = sorted(files)[-1]
print(latest_snap)

In [ ]:
run_desc, training_options = DeepFAMS.setup_training_options(
    gpus       = 2,
    snap       = 1,
    data       = DATA_CUSTOM_DIR,
    resume     = latest_snap
)

In [ ]:
DeepFAMS.utils.execute('nvidia-smi')

In [ ]:
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=True, run_desc=run_desc, training_options=training_options)

In [ ]:
tf.compat.v1.disable_eager_execution()
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=False, run_desc=run_desc, training_options=training_options)